<a href="https://colab.research.google.com/github/karim-mammadov/Kaggle-Datasets-MyMLProject/blob/main/math_students_lahiye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download adilshamim8/math-students

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/math-students.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("/content/math-students.zip")
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.corr(numeric_only=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_corr = df.corr(numeric_only=True)
df_corr

plt.figure(figsize=(10,10))
sns.heatmap(df_corr, annot=True)

In [ ]:
plt.pie(df['sex'].value_counts(), labels=df['sex'].unique(), autopct='%1.1f%%')
plt.show();

In [ ]:
plt.pie(df['Mjob'].value_counts(), labels=df['Mjob'].unique(), autopct='%1.1f%%')
plt.show();

In [ ]:
plt.pie(df['Fjob'].value_counts(), labels=df['Fjob'].unique(), autopct='%1.1f%%')
plt.show();

In [ ]:
df['Mjob'].unique()

In [ ]:
df['Fjob'].unique()

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df['Mjob']=df['Mjob'].str.replace('at_home','At Home')
df['Fjob']=df['Fjob'].str.replace('at_home','At Home')
df

In [ ]:
X=df.drop('G3', axis=1)
y=df['G3'].copy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np

In [ ]:
num_features = X_train.select_dtypes(include=np.number).columns
cat_features = X_train.select_dtypes(exclude=np.number).columns

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

transformer = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
],remainder='passthrough')

estimator = LinearRegression()

full_pipeline = Pipeline([
    ('transformer', transformer),
    ('estimator', estimator)
 ])

In [ ]:
full_pipeline.fit(X_train,y_train)

In [ ]:
full_pipeline.score(X_test,y_test), full_pipeline.score(X_train,y_train)

In [ ]:
y_pred = full_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
r2

In [ ]:
from sklearn.linear_model import Ridge
estimator_1 = Ridge(alpha=0.1)

full_pipeline_1 = Pipeline([
    ('transformer', transformer),
    ('estimator1', estimator_1)
 ])

In [ ]:
full_pipeline_1.fit(X_train,y_train)

In [ ]:
full_pipeline_1.score(X_test,y_test), full_pipeline_1.score(X_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
estimator_2 = RandomForestRegressor(n_estimators=10, random_state=42,min_samples_split=6,max_depth=30)

full_pipeline_2 = Pipeline([
    ('transformer', transformer),
    ('estimator2', estimator_2)
 ])

In [ ]:
full_pipeline_2.fit(X_train,y_train)

In [ ]:
full_pipeline_2.score(X_test,y_test), full_pipeline_2.score(X_train,y_train)

In [ ]:
df_corr_1 = df.corr(numeric_only=True)
df_corr_1

plt.figure(figsize=(10,10))
sns.heatmap(df_corr, annot=True)

In [ ]:
df.corr(numeric_only=True)

In [ ]:
df

In [ ]:
df1_sil = df.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1_sil['Pstatus'] = le.fit_transform(df1_sil['Pstatus'])
df1_sil['school'] = le.fit_transform(df1_sil['school'])
df1_sil['famsize'] = le.fit_transform(df1_sil['famsize'])
df1_sil['sex'] = le.fit_transform(df1_sil['sex'])
df1_sil['Mjob'] = le.fit_transform(df1_sil['Mjob'])
df1_sil['Fjob'] = le.fit_transform(df1_sil['Fjob'])
df1_sil['reason'] = le.fit_transform(df1_sil['reason'])
df1_sil['guardian'] = le.fit_transform(df1_sil['guardian'])
df1_sil

In [ ]:
def remove_outliers_iqr(df1_sil):
  for col in df1_sil.select_dtypes(include=np.number).columns:
      Q1 = df1_sil[col].quantile(0.25)
      Q3 = df1_sil[col].quantile(0.75)
      IQR = Q3 - Q1
      lower_bound = Q1 - 1.5 * IQR
      upper_bound = Q3 + 1.5 * IQR
      df1_sil = df1_sil[(df1_sil[col] >= lower_bound) & (df1_sil[col] <= upper_bound)]
  return df1_sil
df1_temiz = remove_outliers_iqr(df1_sil)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X = df1_temiz.drop('G3', axis=1)
y = df1_temiz['G3'].copy()

In [ ]:
categorical_cols = X.select_dtypes(include="object").columns
numeric_cols = X.select_dtypes(include="number").columns

In [ ]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 42,test_size=0.2)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)

full_pipeline_3 = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(random_state=42))
])

In [ ]:
full_pipeline_3.fit(X_train, y_train)

In [ ]:
full_pipeline_3.score(X_test,y_test), full_pipeline_3.score(X_train,y_train)

In [ ]:
feature_importances = full_pipeline_3.named_steps['model'].feature_importances_

In [ ]:
columns = numeric_cols.tolist() + list(full_pipeline_3.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_cols))

In [ ]:
feature_importances_df = pd.DataFrame({
    'Feature': columns,
    'Importance': feature_importances
    })

In [ ]:
feature_importances_df = feature_importances_df.sort_values(by='Importance',ascending=False)

In [ ]:
feature_importances_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
qrafik1 = feature_importances_df.head()

plt.figure(figsize=(12,6))
sns.barplot(data=qrafik1, x='Importance', y='Feature', hue='Feature', palette='viridis', legend=False)
plt.title('En Yuksek',fontsize=20)
plt.xlabel('Derece')
plt.ylabel('Xususiyyeti')
plt.tight_layout()
plt.show();